In [12]:
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path
import os

env_path = Path(".") / "env"
load_dotenv(dotenv_path=env_path)

openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key="OPENAI_API_KEY")

In [13]:
import librosa
import soundfile as sf
import numpy as np
from pydub import AudioSegment
from pydub.silence import split_on_silence, detect_nonsilent
import os

In [14]:
def audio_splitting(audio_path, output_dir, target_duration=20, max_duration=30, min_duration=5, silence_thresh=40):
    print("Voice File Downloading...")
    audio = AudioSegment.from_file(audio_path)
    total_duration = len(audio)/1000

    print(f"Total Time: {total_duration:.2f} seconds")

    target_duration_ms = target_duration * 1000
    max_duration_ms = max_duration * 1000
    min_duration_ms = min_duration * 1000

    nonsilent_ranges = detect_nonsilent(
        audio, 
        min_silence_len=500,  # 0.5 saniye minimum sessizlik
        silence_thresh=silence_thresh
    )

    if not nonsilent_ranges:
        nonsilent_ranges = [(0, len(audio))]

    chunks = []
    current_chunk_start = 0
    
    for start, end in nonsilent_ranges:
        # Mevcut parça çok uzunsa böl
        while (end - current_chunk_start) > max_duration_ms:
            # Hedef noktaya en yakın sessizlik bul
            search_start = current_chunk_start + target_duration_ms - 2000  # 2 saniye tolerans
            search_end = current_chunk_start + target_duration_ms + 2000
            
            # Bu aralıkta sessizlik ara
            best_split_point = find_best_split_point(
                audio[search_start:search_end], 
                search_start, 
                silence_thresh
            )
            
            if best_split_point:
                chunks.append((current_chunk_start, best_split_point))
                current_chunk_start = best_split_point
            else:
                # Sessizlik bulunamazsa sabit nokta kullan
                chunks.append((current_chunk_start, current_chunk_start + target_duration_ms))
                current_chunk_start += target_duration_ms
        
        # Son parçayı ekle
        if (end - current_chunk_start) >= min_duration_ms:
            chunks.append((current_chunk_start, end))
            current_chunk_start = end

    
    # Parçaları dosya olarak kaydet
    os.makedirs(output_dir, exist_ok=True)
    chunk_files = []
    
    print(f"{len(chunks)} Sound Chunks Was Created")

    for i, (start, end) in enumerate(chunks):
        chunk = audio[start:end]
        duration_sec = len(chunk) / 1000
        
        if duration_sec >= min_duration:
            chunk_file = os.path.join(output_dir, f"chunk_{i:04d}.wav")
            chunk.export(chunk_file, format="wav", parameters=["-ar", "16000"])  # 16kHz sampling
            
            chunk_info = {
                'file': chunk_file,
                'start_time': start / 1000,
                'end_time': end / 1000,
                'duration': duration_sec,
                'chunk_id': i
            }
            chunk_files.append(chunk_info)
    
    print(f"{len(chunk_files)} Certain Voice Chunks Was Created")
    return chunk_files
    